In [5]:
! pip install requests
! pip install beautifulsoup4
! pip install lxml
! pip install pandas
import requests
from bs4 import BeautifulSoup
import lxml
import time
import pandas as pd

In [ ]:
stars = input("스타수 입력 : ")
url = "https://github.com/search?p=1&q=stars%3A%3E{}&type=Repositories".format(stars)

def crawling_func(url):
    print(url)
    try:
        res = requests.get(url,headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text,"lxml")
        p = soup.find_all("a",attrs={"class":"v-align-middle"}) 
    except:
        time.sleep(1)
        crawling_func(url) # 오류를 대비하기위해 재귀함수 호출
        pass
    finally:
        return p
    
topic_ads = []
pages = int(input("검색할 페이지 수(Ex : 10) : "))
print("{}개 이상의 star수를 가진 Repository의 상위 {}페이지를 크롤링합니다. ".format(stars,pages))
print()
for i in range(1,pages+1):# x 페이지까지 탐색
    url = "https://github.com/search?p={}&q=stars%3A%3E10000&type=Repositories".format(i) # 페이지 formatting
    time.sleep(15)  # 10초를 쉬어도 오류가 떴었음
    for j in crawling_func(url):
        topic_ads.append(j.get_text())
        print(j.get_text())

        
topics_dic = {}
topics_list = []
for ad in topic_ads:
    url_topic = "https://github.com/" + ad
    res_topic = requests.get(url_topic,headers=headers)
    res_topic.raise_for_status()
    soup_topic = BeautifulSoup(res_topic.text,"lxml")
    
    topic = soup_topic.find("div",attrs={"class":"BorderGrid-cell"}).find_all("a",attrs={"class":"topic-tag topic-tag-link"})
    project_topics=[i.get_text().replace("\n","").replace("\t","").strip() for i in topic]
    
    # 스타 수(북마크 수, 인기도)
    star_num = soup_topic.find("ul",attrs={"class":"pagehead-actions flex-shrink-0 d-none d-md-inline"}).find("a",attrs={"class":"social-count js-social-count"}).get_text()
    star_num = star_num.replace('\t', '').replace('\n', '').strip()

    topics_list.append([ad,project_topics,star_num])
    
    for t in project_topics:
        if t in topics_dic:
            topics_dic[t] += 1
        else:
            topics_dic[t] = 1

# value로 내림차순 정렬
topics_dic= sorted(topics_dic.items(), key=lambda x: x[1], reverse=True)
df_topic2 = pd.DataFrame(topics_list,columns=['project_name','topic_keyword','star_number'])
# 엑셀로 저장하기

df_topic2.to_excel('Topics_stars{}_project_keyword.xlsx'.format(stars),index=False)
print()
print("star 수가 {} 개 이상인 프로젝트 상위 {}페이지에대한 Data가 저장되었습니다.")
print("종료 .")
